In [1]:
import requests
import json
import pandas as pd
r = requests.get(url = 'https://api.sledilnik.org/api/stats')
resp_text = r.text
json_dict=json.loads(resp_text)

In [2]:
def getCasesOnDate(age_group_string, j, type, casesPerAgeToDate, casesPerAgeToYesterday):
    casesPerAgeToDateCnt = 0
    casesPerAgeToYesterdayCnt = 0
    
    if(casesPerAgeToDate[j][type] is not None):
        casesPerAgeToDateCnt = casesPerAgeToDate[j][type]
    
    if(casesPerAgeToYesterday[j][type] is not None):
        casesPerAgeToYesterdayCnt = casesPerAgeToYesterday[j][type]
        
    casesOnDate =  casesPerAgeToDateCnt - casesPerAgeToYesterdayCnt
#     if(casesOnDate > 0):
#         print(age_group_string + ' type: ' + type + ' casesOnDate: ' + str(casesOnDate))
    return casesOnDate

In [3]:
def populateDataset(ppl_dataset, date, age_group, on_date_female, on_date_male, deceased):
#     print('Date: ' + date + ' age_group: ' + str(age_group) + ' Deceased: ' + str(deceased) + ' Female cases: ' + str(on_date_female) + ' Male cases: ' + str(on_date_male))
    for k in range(on_date_female):
        ppl_dataset[len(ppl_dataset)+1] = {
            'date': date,
            'age_group': age_group,
            'gender': 0,
            'deceased': deceased
        }
    for k in range(on_date_male):
        ppl_dataset[len(ppl_dataset)+1] = {
            'date': date,
            'age_group': age_group,
            'gender': 1,
            'deceased': deceased
        }    
    return ppl_dataset;

In [4]:
def populateDatasetRecovered(ppl_dataset, date, num_recovered):
    for k in range(num_recovered):
        ppl_dataset[len(ppl_dataset)+1] = {
            'date': date,
            'deceased': 0 
        }
    return ppl_dataset;

In [5]:
dates = []
dataset = dict()
deceased_by_age = dict()
ppl_dataset = dict()
switcher = {
    "0-4": 1,
    "5-14": 1,
    "15-24": 2,
    "25-34": 3,
    "35-44": 4,
    "45-54": 5,
    "55-64": 6,
    "65-74": 7,
    "75-84": 8,
    "85-None": 9
}
notDeceasedToDate = 0 #used to calculate notDeceasedToday
for i in range(len(json_dict)):
#     print(json_dict[i])
    
    day = json_dict[i]['day']
    month = json_dict[i]['month']
    year= json_dict[i]['year']
    date = str(day) + '/' + str(month) + '/' + str(year)
#     print('Filtering date: ' + date);
    
    deceasedPerAgeToYesterday = json_dict[i-1]['deceasedPerAgeToDate']
    deceasedPerAgeToDate = json_dict[i]['deceasedPerAgeToDate']
    activePerAgeToYesterday = json_dict[i-1]['statePerAgeToDate']
    activePerAgeToDate = json_dict[i]['statePerAgeToDate'] 
#     recoveredToDate = json_dict[i]['cases']['recoveredToDate']
    closedToDate = json_dict[i]['cases']['closedToDate']

#   format by age_groups
    age_groups = dict() # for date
    for j in range(len(deceasedPerAgeToDate)):
        age_group_string = str(deceasedPerAgeToDate[j]['ageFrom']) + '-' + str(deceasedPerAgeToDate[j]['ageTo'])
        age_group = switcher[age_group_string]
        
        if(i==0):
            deceased_on_date_total, deceased_on_date_female, deceased_on_date_male, not_deceased_on_date_total, not_deceased_on_date_female, not_deceased_on_date_male = 0, 0, 0, 0, 0, 0;
        else:
            deceased_on_date_male = getCasesOnDate(age_group_string, j, 'maleToDate', deceasedPerAgeToDate, deceasedPerAgeToYesterday)
            deceased_on_date_female = getCasesOnDate(age_group_string, j, 'femaleToDate', deceasedPerAgeToDate, deceasedPerAgeToYesterday)
            deceased_on_date_total = getCasesOnDate(age_group_string, j, 'allToDate', deceasedPerAgeToDate, deceasedPerAgeToYesterday)
            active_on_date_male = getCasesOnDate(age_group_string, j, 'maleToDate', activePerAgeToDate, activePerAgeToYesterday)
            active_on_date_female = getCasesOnDate(age_group_string, j, 'femaleToDate', activePerAgeToDate, activePerAgeToYesterday)
            active_on_date_total = getCasesOnDate(age_group_string, j, 'allToDate', activePerAgeToDate, activePerAgeToYesterday)
            # not deceased = active - deceased
            not_deceased_on_date_male = active_on_date_male - deceased_on_date_male
            not_deceased_on_date_female = active_on_date_female - deceased_on_date_female
            not_deceased_on_date_total = active_on_date_total - deceased_on_date_total
        age_groups[age_group_string] = {
            'age_group': age_group,
            'deceased_on_date_male': deceased_on_date_male,
            'deceased_on_date_female': deceased_on_date_female,
            'deceased_on_date_total': deceased_on_date_total,
            'not_deceased_on_date_male': not_deceased_on_date_male,
            'not_deceased_on_date_female': not_deceased_on_date_female,
            'not_deceased_on_date_total': not_deceased_on_date_total 
        }
        
#         if(closedToDate is None):
#             closedToDate = 0
#         notDeceasedToday = closedToDate - notDeceasedToDate
#       create mock_ppl dataset
        if(not_deceased_on_date_total > 0):
            ppl_dataset = populateDataset(ppl_dataset, date, age_group, not_deceased_on_date_female, not_deceased_on_date_male, 0)
#         if(notDeceasedToday > 0):    
#             ppl_dataset = populateDatasetRecovered(ppl_dataset, date, notDeceasedToday)
        if(deceased_on_date_total > 0):
            ppl_dataset = populateDataset(ppl_dataset, date, age_group, deceased_on_date_female, deceased_on_date_male, 1)
        
        notDeceasedToDate = closedToDate

In [39]:
print(ppl_dataset);

{1: {'date': '14/3/2020', 'deceased': 0}, 2: {'date': '19/3/2020', 'deceased': 0}, 3: {'date': '19/3/2020', 'deceased': 0}, 4: {'date': '19/3/2020', 'deceased': 0}, 5: {'date': '19/3/2020', 'deceased': 0}, 6: {'date': '19/3/2020', 'deceased': 0}, 7: {'date': '20/3/2020', 'deceased': 0}, 8: {'date': '20/3/2020', 'deceased': 0}, 9: {'date': '20/3/2020', 'deceased': 0}, 10: {'date': '20/3/2020', 'deceased': 0}, 11: {'date': '21/3/2020', 'deceased': 0}, 12: {'date': '21/3/2020', 'deceased': 0}, 13: {'date': '21/3/2020', 'deceased': 0}, 14: {'date': '21/3/2020', 'deceased': 0}, 15: {'date': '21/3/2020', 'deceased': 0}, 16: {'date': '21/3/2020', 'deceased': 0}, 17: {'date': '22/3/2020', 'deceased': 0}, 18: {'date': '22/3/2020', 'deceased': 0}, 19: {'date': '22/3/2020', 'deceased': 0}, 20: {'date': '22/3/2020', 'deceased': 0}, 21: {'date': '23/3/2020', 'deceased': 0}, 22: {'date': '23/3/2020', 'deceased': 0}, 23: {'date': '23/3/2020', 'deceased': 0}, 24: {'date': '23/3/2020', 'deceased': 0}, 

In [6]:
df = pd.DataFrame(ppl_dataset).T 
df

,date,age_group,gender,deceased
1,4/3/2020,6,1,0
2,5/3/2020,3,1,0
3,5/3/2020,3,1,0
4,5/3/2020,4,1,0
5,5/3/2020,5,0,0
...,...,...,...,...
14921,19/10/2020,9,1,0
14922,19/10/2020,9,1,0
14923,19/10/2020,9,1,0
14924,19/10/2020,9,1,0


In [41]:
df.to_csv("slovenia_mock_people.csv")